<h1 align="center">CHOPS TRANSFORM</h1>

In [1]:
from PIL import Image, ImageChops, ImageDraw
from utils import prep_and_chops

## 1 - CHOPS EXAMPLES

In [4]:
# --- supposer que prep_and_chops est déjà définie dans ton code ---
# from your_module import prep_and_chops

A = Image.open("data/world.jpg")
B = Image.open("data/network.png")

def save(img, name):
    img.save(name); print("→", name)

# Petit utilitaire: récupérer juste la paire préparée via prep_and_chops
def prepped_pair(img1, img2, **prep_kwargs):
    # on force resize partout
    prep_kwargs.setdefault("size_strategy", "resize")
    prep_kwargs.setdefault("resample", Image.LANCZOS)
    A2, B2, _ = prep_and_chops(img1, img2, op="difference", **prep_kwargs)
    return A2, B2

# 1) add : ((A + B)/scale + offset)
_, _, add1 = prep_and_chops(A, B, op="add", scale=1.0, offset=0,
                            size_strategy="resize", resample=Image.LANCZOS)
save(add1, "chops/ex_add.png")
_, _, add2 = prep_and_chops(A, B, op="add", scale=2.0, offset=0,
                            size_strategy="resize", resample=Image.LANCZOS)
save(add2, "chops/ex_add_scale2.png")

# 2) add_modulo : (A + B) % MAX
_, _, addm = prep_and_chops(A, B, op="add_modulo",
                            size_strategy="resize", resample=Image.LANCZOS)
save(addm, "chops/ex_add_modulo.png")

# 3) blend : alias Image.blend (on prépare d’abord)
A2, B2 = prepped_pair(A, B, mode="RGB")
blend_30 = ImageChops.blend(A2, B2, alpha=0.3)
save(blend_30, "chops/ex_blend_0_3.png")

# 4) composite : alias Image.composite (masque L)
A2, B2 = prepped_pair(A, B, mode="RGB")
mask = Image.new("L", A2.size, 0)
ImageDraw.Draw(mask).rectangle([A2.width//4, A2.height//4, 3*A2.width//4, 3*A2.height//4], fill=255)
comp = ImageChops.composite(A2, B2, mask)  # blanc = A2, noir = B2
save(comp, "chops/ex_composite_mask.png")

# 5) constant : aplat de gris (même mode/taille que A2)
A2, _ = prepped_pair(A, B, mode="RGB")
const = ImageChops.constant(A2, 64)
save(const, "chops/ex_constant_64.png")

# 6) darker : min(A,B)
_, _, dark = prep_and_chops(A, B, op="darker",
                            size_strategy="resize", resample=Image.LANCZOS)
save(dark, "chops/ex_darker.png")

# 7) difference : |A - B|
_, _, diff = prep_and_chops(A, B, op="difference",
                            size_strategy="resize", resample=Image.LANCZOS)
save(diff, "chops/ex_difference.png")

# 8) duplicate : alias .copy()
A2, _ = prepped_pair(A, B, mode="RGB")
dup = ImageChops.duplicate(A2)
save(dup, "chops/ex_duplicate.png")

# 9) invert : MAX - image
A2, _ = prepped_pair(A, B, mode="RGB")
inv = ImageChops.invert(A2)
save(inv, "chops/ex_invert.png")

# 10) lighter : max(A,B)
_, _, light = prep_and_chops(A, B, op="lighter",
                             size_strategy="resize", resample=Image.LANCZOS)
save(light, "chops/ex_lighter.png")

# 11) logical_* : exigent le mode "1"
def to_bin(img, thresh=128):
    g = img.convert("L")
    return g.point(lambda p: 255 if p >= thresh else 0, mode="1")

A2, B2 = prepped_pair(A, B)  # déjà en resize
A1, B1 = to_bin(A2), to_bin(B2)

land = ImageChops.logical_and(A1, B1)
save(land.convert("L"), "chops/ex_logical_and.png")

lor  = ImageChops.logical_or(A1, B1)
save(lor.convert("L"), "chops/ex_logical_or.png")

lxor = ImageChops.logical_xor(A1, B1)
save(lxor.convert("L"), "chops/ex_logical_xor.png")

# 12) multiply : A * B / MAX
_, _, mul = prep_and_chops(A, B, op="multiply",
                           size_strategy="resize", resample=Image.LANCZOS)
save(mul, "chops/ex_multiply.png")

# 13) soft_light / hard_light / overlay : selon ta version de Pillow
def try_chops(name, img1, img2, outname):
    fn = getattr(ImageChops, name, None)
    if fn is None:
        print(f"(skip) {name} non dispo dans cette Pillow")
        return
    I1, I2 = prepped_pair(img1, img2, mode="RGB")
    out = fn(I1, I2)
    save(out, outname)

try_chops("soft_light", A, B, "chops/ex_soft_light.png")
try_chops("hard_light", A, B, "chops/ex_hard_light.png")
try_chops("overlay",    A, B, "chops/ex_overlay.png")

# 14) offset : décalage (wrap)
A2, _ = prepped_pair(A, B, mode="RGB")
off = ImageChops.offset(A2, xoffset=40, yoffset=20)
save(off, "chops/ex_offset.png")

# 15) screen : 1 - (1-A)*(1-B)
_, _, scr = prep_and_chops(A, B, op="screen",
                           size_strategy="resize", resample=Image.LANCZOS)
save(scr, "chops/ex_screen.png")

# 16) subtract : ((A - B)/scale + offset)
_, _, sub1 = prep_and_chops(A, B, op="subtract", scale=1.0, offset=0,
                            size_strategy="resize", resample=Image.LANCZOS)
save(sub1, "chops/ex_subtract.png")
_, _, sub2 = prep_and_chops(A, B, op="subtract", scale=1.0, offset=32,
                            size_strategy="resize", resample=Image.LANCZOS)
save(sub2, "chops/ex_subtract_offset32.png")

# 17) subtract_modulo : (A - B) % MAX
_, _, subm = prep_and_chops(A, B, op="subtract_modulo",
                            size_strategy="resize", resample=Image.LANCZOS)
save(subm, "chops/ex_subtract_modulo.png")

print("Fini ✅")


→ chops/ex_add.png
→ chops/ex_add_scale2.png
→ chops/ex_add_modulo.png
→ chops/ex_blend_0_3.png
→ chops/ex_composite_mask.png
→ chops/ex_constant_64.png
→ chops/ex_darker.png
→ chops/ex_difference.png
→ chops/ex_duplicate.png
→ chops/ex_invert.png
→ chops/ex_lighter.png
→ chops/ex_logical_and.png
→ chops/ex_logical_or.png
→ chops/ex_logical_xor.png
→ chops/ex_multiply.png
→ chops/ex_soft_light.png
→ chops/ex_hard_light.png
→ chops/ex_overlay.png
→ chops/ex_offset.png
→ chops/ex_screen.png
→ chops/ex_subtract.png
→ chops/ex_subtract_offset32.png
→ chops/ex_subtract_modulo.png
Fini ✅
